In [2]:
%pip install langchain-community chromadb sentence-transformers faiss-cpu requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 9.3 MB/s eta 0:00:00ta 0:00:01

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import json
import requests
from typing import List, Dict, Any
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
def get_best_lightweight_model():
    try:
        response = requests.get("http://localhost:11434/api/tags", timeout=10)
        if response.status_code == 200:
            models = response.json().get('models', [])
            available_models = [model['name'] for model in models]
            print("Available models:", available_models)
            preferred_models = ['phi3:mini', 'gemma2:2b', 'qwen2:0.5b', 'phi:latest', 'llama2:latest']
            
            for model in preferred_models:
                if model in available_models:
                    print(f"✅ Using {model} (lightweight)")
                    return model
            
            if available_models:
                print(f"⚠️ Using {available_models[0]} (fallback)")
                return available_models[0]
                
        return None
    except:
        return None

In [13]:
# Simple LLM Test Block
def test_llm_connection():
    """Test if the LLM is working correctly with a simple query"""
    print("🧪 Testing LLM connection...")
    
    # Simple test query
    test_query = "Hello! Please respond with a normal English sentence about the weather today."
    
    print(f"📤 Sending query: '{test_query}'")
    
    try:
        # Get response from LLM
        response = llm.invoke(test_query)
        
        print("📥 Response received:")
        print("-" * 50)
        print(response)
        print("-" * 50)
        
        # Check if response looks normal
        if response and not any(char * 4 in response for char in ['@', '#', '$', '%', '&']):
            if len(response.strip()) > 10:  # Reasonable response length
                print("✅ LLM is working correctly! Response looks normal.")
                return True
            else:
                print("⚠️  LLM responded, but response is very short.")
                return False
        else:
            print("❌ LLM response contains special characters or looks abnormal.")
            print("   This usually indicates a model issue. Try: ollama pull phi3:mini")
            return False
            
    except Exception as e:
        print(f"❌ Error testing LLM: {e}")
        print("   Make sure Ollama is running: ollama serve")
        return False

# Run the test
llm_working = test_llm_connection()

if llm_working:
    print("\n🎉 Great! Your LLM is ready for resume shortlisting.")
else:
    print("\n💡 Troubleshooting tips:")
    print("1. Run: ollama serve (in terminal)")
    print("2. Run: ollama pull phi3:mini (for a good lightweight model)")
    print("3. Check: ollama list (to see available models)")
    print("4. Restart your Python kernel after installing new models")

🧪 Testing LLM connection...
📤 Sending query: 'Hello! Please respond with a normal English sentence about the weather today.'
📥 Response received:
--------------------------------------------------
The sky is clear and sunny, making it an excellent day for outdoor activities.
--------------------------------------------------
✅ LLM is working correctly! Response looks normal.

🎉 Great! Your LLM is ready for resume shortlisting.


In [6]:
class LightweightOllama:
    def __init__(self, model_name="phi3:mini"):
        self.model_name = model_name
        self.base_url = "http://localhost:11434"
    
    def invoke(self, prompt):
        try:
            response = requests.post(
                f"{self.base_url}/api/generate",
                json={
                    "model": self.model_name,
                    "prompt": prompt,
                    "stream": False,
                    "options": {
                        "temperature": 0.1,      # Lower temperature for more focused responses
                        "num_predict": 200,      # Limit response length
                        "top_k": 20              # Reduce computation
                    }
                },
                timeout=30
            )
            
            if response.status_code == 200:
                result = response.json().get("response", "")
                # Clean up any special characters
                if "@@@@" in result or "####" in result:
                    return "Model response contained invalid characters. Please try a different model."
                return result
            else:
                return f"Error: {response.status_code}"
                
        except Exception as e:
            return f"Connection error: {str(e)}"

In [ ]:
model_name = get_best_lightweight_model()
if model_name:
    llm = LightweightOllama(model_name)
else:
    # Fallback
    class SimpleLLM:
        def invoke(self, prompt):
            return "Please install a lightweight model: ollama pull phi3:mini"
    llm = SimpleLLM()

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2" 
)

Available models: ['qwen2:0.5b', 'gemma2:2b', 'phi3:mini', 'phi:latest', 'llama2:latest']
✅ Using phi3:mini (lightweight)


/var/folders/py/4k88vyb50_534cj29tc3l25c0000gn/T/ipykernel_77903/2560659210.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
resumes = [
    {
        "id": 1, "name": "Sarah Chen", "email": "sarah.chen@email.com",
        "experience": "Senior Data Scientist at TechCorp (2020-Present): Led ML projects, NLP models. Data Scientist at DataWorks (2018-2020): Recommendation systems, deep learning.",
        "education": "MS Computer Science, Stanford University (2018). BS Mathematics, UC Berkeley (2016).",
        "skills": "Python, Machine Learning, TensorFlow, PyTorch, NLP, SQL",
        "certifications": "AWS ML Specialist, Google Cloud Data Engineer"
    },
    {
        "id": 2, "name": "Michael Rodriguez", "email": "m.rodriguez@email.com",
        "experience": "ML Engineer at AI Innovations (2019-Present): Computer vision, MLOps. Software Engineer at SoftTech (2017-2019): Backend systems, Docker, Kubernetes.",
        "education": "MEng Artificial Intelligence, MIT (2017). BS Computer Science, MIT (2015).",
        "skills": "Python, TensorFlow, Computer Vision, Docker, Kubernetes, MLOps",
        "certifications": "TensorFlow Certified, Kubernetes Administrator"
    },
    {
        "id": 3, "name": "Emily Thompson", "email": "e.thompson@email.com", 
        "experience": "Data Engineer at DataFlow (2019-Present): Data pipelines, Spark optimization. Software Developer at CodeCraft (2017-2019): ETL processes, data integration.",
        "education": "MS Data Science, University of Washington (2017). BS Computer Science, UT Austin (2015).",
        "skills": "Python, SQL, Spark, Hadoop, ETL, AWS",
        "certifications": "Google Cloud Data Engineer, AWS Solutions Architect"
    }
]

In [9]:
def create_lightweight_documents(resume_data):
    documents = []
    for resume in resume_data:
        text = f"{resume['name']} - {resume['email']}. Experience: {resume['experience']} Education: {resume['education']} Skills: {resume['skills']}"
        metadata = {"id": resume['id'], "name": resume['name']}
        documents.append({"page_content": text, "metadata": metadata})
    return documents


In [10]:

documents = create_lightweight_documents(resumes)

try:

    texts = [doc["page_content"] for doc in documents]
    metadatas = [doc["metadata"] for doc in documents]
    
    vectorstore = FAISS.from_texts(texts, embeddings, metadatas=metadatas)
    print("✅ FAISS vector store created (memory efficient)")
except Exception as e:
    print(f"❌ Vector store creation failed: {e}")
    class SimpleRetriever:
        def get_relevant_documents(self, query):
            query_words = set(query.lower().split())
            results = []
            for doc in documents:
                doc_words = set(doc["page_content"].lower().split())
                score = len(query_words.intersection(doc_words)) / len(query_words.union(doc_words)) if query_words else 0
                results.append((score, doc))
            results.sort(key=lambda x: x[0], reverse=True)
            return [doc for score, doc in results[:3]]
    
    retriever = SimpleRetriever()
else:
    retriever = vectorstore.as_retriever(search_kwargs={"k": 2})  

✅ FAISS vector store created (memory efficient)


In [12]:
def shortlist_candidates(query):
    try:
        if hasattr(retriever, 'get_relevant_documents'):
            docs = retriever.get_relevant_documents(query)
        else:
            docs = retriever.invoke(query)
        
        context = "\n\n".join([doc.page_content if hasattr(doc, 'page_content') else doc["page_content"] for doc in docs])
        
        prompt = f"""You are an expert recruiter. Analyze these candidates and recommend the best one.

Candidates:
{context}

Job Requirements: {query}

Please provide a concise analysis (2-3 sentences) and recommend the best candidate:"""
        
        # Get response
        response = llm.invoke(prompt)
        return response
        
    except Exception as e:
        return f"Error: {str(e)}"
print("\n🧪 Testing with lightweight model...")
query = "Find candidates with Python and machine learning experience"
result = shortlist_candidates(query)
print("📋 Results:")
print(result)

print("\n🧪 Test 2: Cloud experience")
query2 = "Looking for candidates with AWS or cloud experience"
result2 = shortlist_candidates(query2)
print("📋 Results:")
print(result2)


🧪 Testing with lightweight model...


/var/folders/py/4k88vyb50_534cj29tc3l25c0000gn/T/ipykernel_77903/349406286.py:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)


📋 Results:
Both Sarah Chen and Michael Rodriguez have relevant experiences in Machine Learning; however, considering your need for NLP expertise as well, I would recommend Sarah Chen. Her background with Natural Language Processing models complements her ML experience at TechCorp better than Michael's computer vision focus.

🧪 Test 2: Cloud experience
📋 Results:
Based on your requirements for AWS or cloud expertise, I would recommend Emily Thompson. Her skills include Spark optimization which is closely related to big data processing in clouds like Amazon EMR, as well as her experience with Hadoop and AWS services during her time at CodeCraft. Although Michael Rodriguez has MLOps experience that often involves using the cloud for machine learning workflows, Emily's specific skills align more directly with your needs.
